#### How to run this notebook
For simplicity, just run all cells
- open browser and go to: localhost:7863
- since the actual KG data is large, i cant upload to the github, i have created smaller kg_test with only 50 nodes.[first 2 chapters from the norwegian index.]

In [ ]:
from src.model import get_llamaindex_model, get_llamaindex_model_mini, get_huggingface_embedding_model
from llama_index.core import Settings

llm = get_llamaindex_model_mini()

llm2 = get_llamaindex_model()

embed_model = get_huggingface_embedding_model()
Settings.embed_model = embed_model
Settings.llm = llm

C:\Users\newac\OneDrive\Desktop\Master\final_structure\src\__init__.py:16: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from .DAG_creator import build_rag_workflow


In [2]:
from src import get_azure_openai_model, get_azure_openai_chat_model, get_azure_openai_mini_model

model = get_azure_openai_chat_model()
model_mini = get_azure_openai_mini_model()

In [ ]:
from src.parser import markdownParser
nodes = markdownParser(input_dir="./kgdata/")
print(f"Processed {len(nodes)} nodes.")


Processed 945 nodes.


In [ ]:
nodes[5]  # Display a slice of the nodes to verify processing

In [ ]:
from src.graphflow_v2 import build_emergency_rag_workflowv2

##### Possible UI

In [ ]:
import gradio as gr
import pprint
from io import StringIO
import sys
import concurrent.futures
import threading
import time
import json

def build_emergency_rag_interface(build_emergency_rag_workflow, model, nodes):
    """
    Creates a Gradio interface for the Emergency Dispatch + RAG workflow
    
    Args:
        build_emergency_rag_workflow: Function that builds the emergency RAG workflow
        model: The language model to use
        nodes: Processed nodes from markdownParser
    """
    
    def run_single_retriever(question, call_transcript, retriever_type, load_persist_vector, load_persist_kg):
        """Run emergency RAG workflow for a single retriever type"""
        try:
            print(f"Starting {retriever_type} retriever...")
            
            # Build the emergency RAG workflow
            app = build_emergency_rag_workflow()
            
            # Set up inputs for emergency RAG
            inputs = {
                "question": question,
                "call_transcript": call_transcript if call_transcript.strip() else None,
                "llm": model,
                "retriever_type": retriever_type,
                "load_persist_vector": load_persist_vector,
                "load_persist_kg": load_persist_kg,
                "nodes": nodes,
                "max_iterations": 2
            }
            
            # Run the workflow and capture outputs
            final_value = None
            for output in app.stream(inputs):
                for key, value in output.items():
                    final_value = value
            
            # Format results
            if final_value:
                # Get basic RAG response
                response = final_value.get('generation', 'No response generated')
#                 documents = final_value.get('documents', [])
                
#                 # Get emergency-specific data
#                 is_emergency = final_value.get('is_emergency', False)
#                 emergency_analysis = final_value.get('emergency_analysis', {})
#                 structured_report = final_value.get('structured_report', {})
#                 emergency_type = final_value.get('emergency_type', 'none')
                
#                 # Format documents
#                 formatted_docs = []
#                 for i, doc in enumerate(documents):
#                     header_path = doc.node.metadata.get('header_path', 'No header path available')
#                     text_content = doc.node.text
#                     source = doc.node.metadata.get('source', 'unknown')
#                     score = getattr(doc, 'score', 'N/A')
                    
#                     formatted_doc = f"""**Document {i+1}** (Score: {score})
# **Header Path:** {header_path}
# **Source:** {source}
# **Content:**
# {text_content}
# {"="*80}
# """
#                     formatted_docs.append(formatted_doc)
                
#                 doc_text = "\n".join(formatted_docs) if formatted_docs else "No documents retrieved."
                
                # Build comprehensive result
                result_parts = []
                
                # # Emergency Analysis Section
                # if is_emergency and emergency_analysis:
                #     result_parts.append("🚨 **EMERGENCY DETECTED** 🚨")
                #     result_parts.append(f"**Emergency Type:** {emergency_type}")
                #     result_parts.append("**Emergency Analysis:**")
                #     result_parts.append(json.dumps(emergency_analysis, indent=2))
                #     result_parts.append("\n" + "="*60 + "\n")
                
                # # Structured Report Section
                # if structured_report:
                #     result_parts.append("📋 **STRUCTURED EMERGENCY REPORT:**")
                #     result_parts.append(json.dumps(structured_report, indent=2))
                #     result_parts.append("\n" + "="*60 + "\n")
                
                # RAG Response Section
                result_parts.append("🤖 **RAG RESPONSE:**")
                result_parts.append(response)
                result_parts.append("\n" + "="*60 + "\n")
                
                # # Documents Section
                # result_parts.append(f"📚 **RETRIEVED DOCUMENTS ({len(documents)}):**")
                # result_parts.append(doc_text)
                
                final_result = "\n\n".join(result_parts)
                print(f"Completed {retriever_type} retriever")
                return final_result
            else:
                return f"**❌ ERROR:** No response generated for {retriever_type}"
                
        except Exception as e:
            error_msg = f"**❌ ERROR in {retriever_type}:** {str(e)}"
            print(f"Error in {retriever_type}: {str(e)}")
            return error_msg

    def process_emergency_question_streaming(question, call_transcript, load_persist_vector, load_persist_kg):
        """Process emergency question with real-time streaming updates"""
        
        if not question.strip():
            yield "Please enter a question.", "Please enter a question.", "Please enter a question."
            return
        
        print(f"🚨 Starting Emergency RAG processing for: {question[:50]}...")
        start_time = time.time()
        
        # Initialize with processing messages  
        results = {
            'vector_store': "🔄 **VECTOR STORE** - Processing emergency query... Please wait.",
            'knowledge_graph': "🔄 **KNOWLEDGE GRAPH** - Processing emergency query... Please wait.", 
            'hybrid': "🔄 **HYBRID** - Processing emergency query... Please wait."
        }
        
        # Yield initial state immediately
        yield results['vector_store'], results['knowledge_graph'], results['hybrid']
        
        # Create thread-safe variables
        results_lock = threading.Lock()
        completion_times = {}
        completed_count = 0
        
        def process_single_result(retriever_type, question, call_transcript, load_persist_vector, load_persist_kg):
            """Process a single retriever and return the result"""
            thread_start = time.time()
            try:
                result = run_single_retriever(question, call_transcript, retriever_type, load_persist_vector, load_persist_kg)
                exec_time = time.time() - thread_start
                enhanced_result = f"⚡ **Completed in {exec_time:.2f}s** | **{retriever_type.replace('_', ' ').title()}**\n\n{result}"
                return retriever_type, enhanced_result, exec_time, True
            except Exception as e:
                exec_time = time.time() - thread_start
                error_result = f"❌ **FAILED in {exec_time:.2f}s** | **{retriever_type.replace('_', ' ').title()}**\n\n**Error:** {str(e)}"
                return retriever_type, error_result, exec_time, False
        
        # Start all three workflows concurrently
        with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
            # Submit all tasks
            futures = {
                executor.submit(process_single_result, 'vector_store', question, call_transcript, load_persist_vector, load_persist_kg): 'vector_store',
                executor.submit(process_single_result, 'knowledge_graph', question, call_transcript, load_persist_vector, load_persist_kg): 'knowledge_graph',
                executor.submit(process_single_result, 'hybrid', question, call_transcript, load_persist_vector, load_persist_kg): 'hybrid'
            }
            
            # Process results as they complete and yield immediately
            for future in concurrent.futures.as_completed(futures):
                try:
                    retriever_type, result, exec_time, success = future.result()
                    
                    with results_lock:
                        completed_count += 1
                        results[retriever_type] = result
                        if success:
                            completion_times[retriever_type] = exec_time
                        
                        print(f"✅ {retriever_type} completed in {exec_time:.2f}s ({completed_count}/3)")
                    
                    # Yield updated results immediately - this is the key!
                    yield results['vector_store'], results['knowledge_graph'], results['hybrid']
                    
                except Exception as e:
                    print(f"❌ Future execution error: {str(e)}")
                    # Still yield the current state even if there's an error
                    yield results['vector_store'], results['knowledge_graph'], results['hybrid']
        
        # Final processing - add trophy to fastest result
        if completion_times:
            fastest_time = min(completion_times.values())
            fastest_retriever = min(completion_times.items(), key=lambda x: x[1])[0]
            
            with results_lock:
                if fastest_retriever in results:
                    results[fastest_retriever] = f"🏆 **FASTEST** | {results[fastest_retriever]}"
        
        total_time = time.time() - start_time
        print(f"🎉 All emergency retrievers completed! Total time: {total_time:.2f}s")
        
        # Final yield with trophy added
        yield results['vector_store'], results['knowledge_graph'], results['hybrid']
    
    # Create Gradio interface
    with gr.Blocks(title="Emergency Dispatch RAG Interface", theme=gr.themes.Soft()) as interface:
        
        gr.Markdown("# 🚨 Emergency Dispatch RAG Workflow Interface")
        gr.Markdown("This is an enhanced Retrieval-Augmented Generation (RAG) interface for **medical emergencies and emergency dispatch**. The system can handle both regular medical queries and emergency call scenarios with structured emergency analysis.")
        gr.Markdown("**⚡ Real-time Emergency Processing:** When you click 'Process Emergency Query', all three retriever types (Vector Store, Knowledge Graph, and Hybrid) run **in parallel** and results appear **immediately** as each one finishes! Emergency scenarios get special analysis and structured reports.")
        
        with gr.Row():
            with gr.Column(scale=2):
                # Input section
                gr.Markdown("### 📝 Input Section")
                
                question_input = gr.Textbox(
                    label="🔍 Question / Query",
                    placeholder="Enter your medical question or emergency query here...",
                    lines=3,
                    value="how to revive a person who is unconscious"
                )
                
                call_transcript_input = gr.Textbox(
                    label="🎙️ Emergency Call Transcript (Optional)",
                    placeholder="Paste emergency call transcript here for detailed emergency analysis...",
                    lines=5,
                    value=""
                )
                
                gr.Markdown("### ⚙️ Configuration")
                
                with gr.Row():
                    load_persist_vector = gr.Textbox(
                        label="📦 Vector Store Path",
                        value="./vectorstore",
                        placeholder="Path to vector store"
                    )
                    
                    load_persist_kg = gr.Textbox(
                        label="🕸️ Knowledge Graph Store Path", 
                        value="./kgstore_test_50",
                        placeholder="Path to knowledge graph store"
                    )
                
                submit_btn = gr.Button("🚨 Process Emergency Query", variant="primary", size="lg")
                clear_btn = gr.Button("🧹 Clear All", variant="secondary")
            
            with gr.Column(scale=3):
                # Output section
                gr.Markdown("### 📊 Emergency Analysis Results")
                
                with gr.Tabs():
                    with gr.TabItem("🏪 Vector Store"):
                        vector_output = gr.Textbox(
                            label="Vector Store Emergency Results",
                            lines=25,
                            max_lines=30,
                            show_copy_button=True
                        )
                    
                    with gr.TabItem("🕸️ Knowledge Graph"):
                        kg_output = gr.Textbox(
                            label="Knowledge Graph Emergency Results",
                            lines=25,
                            max_lines=30,
                            show_copy_button=True
                        )
                    
                    with gr.TabItem("🔄 Hybrid"):
                        hybrid_output = gr.Textbox(
                            label="Hybrid Emergency Results",
                            lines=25,
                            max_lines=30,
                            show_copy_button=True
                        )
        
        # Event handlers with streaming
        submit_btn.click(
            fn=process_emergency_question_streaming,
            inputs=[question_input, call_transcript_input, load_persist_vector, load_persist_kg],
            outputs=[vector_output, kg_output, hybrid_output]
        )
        
        clear_btn.click(
            fn=lambda: ("", "", "", "", "", "", ""),
            outputs=[question_input, call_transcript_input, vector_output, kg_output, hybrid_output, load_persist_vector, load_persist_kg]
        )
        
        # Example sections
        gr.Markdown("### 💡 Example Emergency Queries")
        
        with gr.Row():
            with gr.Column():
                gr.Markdown("**🩺 Medical Emergency Examples:**")
                medical_examples = [
                    "how to revive a person who is unconscious",
                    "what are the steps for CPR?",
                    "how to treat a burn injury?",
                    "what to do in case of choking?",
                    "what are the symptoms of a heart attack?",
                ]
                
                for example in medical_examples:
                    gr.Button(example, size="sm").click(
                        fn=lambda x=example: x,
                        outputs=question_input
                    )
            
            with gr.Column():
                gr.Markdown("**🚨 Emergency Dispatch Examples:**")
                dispatch_examples = [
                    "person collapsed in public place",
                    "car accident with injuries reported", 
                    "house fire with people trapped",
                    "suspected heart attack patient",
                    "motorcycle accident victim unconscious",
                ]
                
                for example in dispatch_examples:
                    gr.Button(example, size="sm").click(
                        fn=lambda x=example: x,
                        outputs=question_input
                    )
        
        gr.Markdown("### 📞 Sample Emergency Call Transcript")
        sample_transcript = """Dispatcher: 911, what's your emergency?
        Caller: Hi, I just found someone unconscious in the park. I need help!
        Dispatcher: Okay, stay calm. Can you tell me your location?
        aller: Yes, I'm at Riverside Park, near the north entrance by the playground.
        Dispatcher: Got it. Are they breathing?
        Caller: Um, I'm not sure. Let me check... No, I don't think so!"""
        
        gr.Button("📋 Use Sample Emergency Call", size="sm").click(
            fn=lambda: sample_transcript,
            outputs=call_transcript_input
        )
        
        gr.Markdown("""
        ### 🔧 How It Works:
        1. **Emergency Detection**: The system automatically detects if your input is emergency-related
        2. **Call Analysis**: If a call transcript is provided, detailed emergency analysis is performed
        3. **Structured Reports**: Emergency scenarios generate structured dispatch reports
        4. **Concurrent RAG**: All retriever types run in parallel for fastest response
        5. **Real-time Updates**: See results as soon as each retriever completes
        """)
    
    return interface

# Usage example:
interface = build_emergency_rag_interface(build_emergency_rag_workflowv2, model, nodes)

# Launch the interface
if __name__ == "__main__":
    interface.launch(
        server_name="0.0.0.0",  # Allow external access
        server_port=7864,       # Different port from original
        share=False,            # Set to True for public sharing
        debug=True              # Enable debug mode
    )

* Running on local URL:  http://0.0.0.0:7864
* To create a public link, set `share=True` in `launch()`.


🚨 Starting Emergency RAG processing for: how to revive a person who is unconscious...
Starting vector_store retriever...
Starting knowledge_graph retriever...
Starting hybrid retriever...
--- DETECT EMERGENCY CONTEXT ---
--- DETECT EMERGENCY CONTEXT ---
--- DETECT EMERGENCY CONTEXT ---
Emergency detected: True, Type: medical
Emergency detected: True, Type: medical
Emergency detected: True, Type: medical
--- ROUTING TO EMERGENCY ANALYSIS ---
--- ROUTING TO EMERGENCY ANALYSIS ---
--- ROUTING TO EMERGENCY ANALYSIS ---
--- ANALYZE EMERGENCY CALL ---
--- ANALYZE EMERGENCY CALL ---
--- ANALYZE EMERGENCY CALL ---
--- GENERATE EMERGENCY REPORT ---
--- GENERATE EMERGENCY REPORT ---
--- GENERATE EMERGENCY REPORT ---
emergency summary: Medical emergency - unconscious person
--- RETRIEVE ---
Using original question: Medical emergency - unconscious person + Unconsciousness
Emergency context: True
final question for retrieval: Medical emergency - unconscious person + Unconsciousness
emergency summar

  0%|          | 0/2 [00:00<?, ?it/s]

Error in vector_store: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'medium'}}}}}
✅ vector_store completed in 47.09s (1/3)


100%|██████████| 2/2 [00:03<00:00,  1.85s/it]


Vector Store Retrieved 15 documents
Knowledge Graph Retrieved 11 documents
Retrieved 26 documents


100%|██████████| 2/2 [00:04<00:00,  2.48s/it]


Retrieved 13 documents
--- CHECK DOCUMENT RELEVANCE TO QUESTION ---
--- CHECK DOCUMENT RELEVANCE TO QUESTION ---
--- GRADE: DOCUMENT RELEVANT ---
--- GRADE: DOCUMENT RELEVANT ---
--- GRADE: DOCUMENT RELEVANT ---
--- GRADE: DOCUMENT RELEVANT ---
--- GRADE: DOCUMENT RELEVANT ---
--- GRADE: DOCUMENT RELEVANT ---
--- GRADE: DOCUMENT RELEVANT ---
--- GRADE: DOCUMENT RELEVANT ---
--- GRADE: DOCUMENT RELEVANT ---
--- GRADE: DOCUMENT RELEVANT ---
--- GRADE: DOCUMENT RELEVANT ---
--- GRADE: DOCUMENT RELEVANT ---
--- GRADE: DOCUMENT RELEVANT ---
--- GRADE: DOCUMENT RELEVANT ---
Error in hybrid: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'Res